# Como funciona

Se debe tener al menos 2 telefonos. El telefono inicial que es el que va a despertar a los demas. Este siempre va a estar sensando el movimiento.
La detección de movimiento se hizo por medio del sensor de luz y tomando un valor de referencia al instalar el telefono en su posición fija. Para determinar el rango de luz se hicieron pruebas de movimientos frente al equipo y a una distancia de por lo menos 2 metros.
Una vez que detecta movimiento le envia un sms al siguiente telefono para que despierte. Este último despierta y empieza a sensar, cuando detecta movimiento empiezaa a tomar 5 fotos y las compara mostrando la foto original y la foto donde encontro movimiento, así sucesivamente con los demas dispositivos

# Codigo

In [ ]:
%matplotlib inline
!adb forward tcp:9999 tcp:47048
import android
import matplotlib.pyplot as plt
import sys, os
import numpy as np
import cv2
import android
import json
import time
import random
#Importar sleep
from time import sleep
#Se crea la instancia del objeto android
droid=android.Android()
print "Start..."

droid = android.Android()

#Se toma una muestra del tiempo actual, ya que esta variable se utilizará mas adelante para la busqueda de los SMSs
lastcheck = int(round(time.time() * 1000))
print lastcheck
#Se solicita el número de teléfono del que se recibiran las notificaciones para despertar a este dispositovo, 
#SI este teléfono va a ser el primero en la línea de detección, este valor debe dejarse vacio.
vecino1 = droid.dialogGetInput('Detector', 'Introduce cual es el telefono del que esperas avisos, si eres el primer telefono en la linea de vigilancia deja la casilla vacia').result
#Se solicita el número de teléfono al cual se le enviará la notificación para despertarlo
vecino2 = droid.dialogGetInput('Detector', 'Introduce cual es el telefono al que vas a notificar').result

#Se define el modo de ejecución de este dispositivo, para determinar si será  VIGILANTE o DURMIENTE,
if not vecino1:
    modo = 'V' #Vigilante desde el principio se encuentra detectando movimiento es el primer dispositivo en la línea de detección
else:
    modo = 'D' #Durmiente esta a la espera de un sms para iniciar la detección de movimiento
print modo
mensaje = "Move detected wake up..." #Se especifica el mensaje que se enviará para comunicar los dispositivos

droid.cameraCapturePicture('/sdcard/DCIM/CAMERA/SENSOR/foto_original.jpg',True)#,1,1)
droid.startSensingTimed(4,25)
droid.ttsSpeak("Empezando deteccion de luz")
counter = 0
counter2 = 0
counter3 = 0

def sendSMS():
    droid.smsSend(vecino2,mensaje)#Envia un SMS se manda el número de teléfono y el mensaje
    droid.ttsSpeak("Enviando sms")

def detectMove(counter,counter2,counter3):
    ##Codigo de detección de movimiento
    ##Debe devolver un booleano True o False
    while counter == 0:
        luz = droid.sensorsGetLight()
        infoIluminacion = " Iluminacion: %s" %luz[1]
        #print infoIluminacion
        if luz[1]<28:
            droid.ttsSpeak("Hubo movimiento")
            print 'Hubo movimiento'
            counter+=1
            droid.stopSensing()

            droid.ttsSpeak('Tomando foto')

            while counter2 <= 5:
                droid.cameraCapturePicture('/sdcard/DCIM/CAMERA/SENSOR/foto_'+str(counter2)+'.jpg',True)#,1,1)
                counter2 +=1

            print 'Hecho sin errores...'

            droid.ttsSpeak('Terminado sin errores...')
            os.system('adb pull /sdcard/DCIM/CAMERA/SENSOR/ ./')

            droid.ttsSpeak('Empezando comparacion')
            pic = os.path.join('.','foto_original.jpg')
            img = cv2.imread(pic, 0)

            while counter3 <= 5:

                #compara imagenes
                pic2 = os.path.join('.','foto_'+str(counter3) +'.jpg')
                img2 = cv2.imread(pic2, 0)

                compare_images(img,img2,'titulo')
                valor = mse(img,img2)
                if valor>1000 :
                    print 'hubo movimiento'
                    #droid.ttsSpeak('Comprobado hubo movimiento')
                    #counter3 +=5
                    return True
                counter3 +=1

            return False


def scanSMSs(lastcheck):
    #Obtiene la lista de mensajes recibidos por el dispositivo
    msgs = droid.smsGetMessages(False, 'inbox')
    #Se obtienen solo la sección de mensajes
    m = str(msgs[1])
    #Se eliminan caracteres especiales y se cambian las comillas simples por comillas dobles para poder hacer el parse de json
    m = m.replace('\\','-').replace('"','<').replace("u'",'"').replace("'",'"')
    #print m
    #Se hace el parse de json a una lista de objetos
    d = json.loads(m)
    #Se inicializa un contador, se aumentará cada que se encuentre un mensaje
    count = 1
    #Se inicializa una bandera para determinar si se recibio el mensaje para despertar al dispositivo o no.
    startwatch = False
    #Se realiza una iteración de todos los mensajes
    for msg in d:
        #Se compara que la fecha del mensaje sea mayor a la fecha de ultima verificación <lastcheck>.
        #Se compara que el remitente del mensaje sea el telefono especificado como vecino1
        #Se compara que el sms contenga el mensaje clave para desperar al dispositivo
        if long(msg['date']) > lastcheck and msg['address'] == vecino1 and mensaje in str(msg['body']):
            #Si se detecta un mensaje se actualiza la bandera para que inicie a sensar, y se actualiza el tiempo de última verificación
            print "Start watch..."
            startwatch = True
            lastcheck = int(round(time.time() * 1000))
            #Se elimina el mensaje recibido
            droid.smsDeleteMessage(msg['_id'])
            #Se detiene el iterador de mensajes
            break;
        count += 1
    print count
    #en caso de que no se haya detectado mensaje solo se muestra en consola que seguira durmiendo 
    if not startwatch:
        print "Go to sleep"
    return startwatch

#esta ponerla aparte es para la comparacion de imagenes----------------------------------------------------------------
#Se utiliza el siguiente código para la comparación de las imagenes una vez que estan en el equipo. 
#Éste hay que ejecutarlo antes de empezar la detección de movimiento.
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err
 
def compare_images(imageA, imageB, title):
    # compute the mean squared error and structural similarity
    # index for the images
    m = mse(imageA, imageB)
    # s = ssim(imageA, imageB)
 
    # setup the figure
    fig = plt.figure(title)
    #plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
 
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")
 
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")
 
    # show the images
    plt.show()
    print m

    
#Este contador medirá cuantas veces se intento detectar movimiento sin éxito
counterdetect = 0
#Inicia la ejecución del programa en un programa infinito
while(True):
    #Si el modo es VIGILANTE
    if modo == 'V':
        #Inicia la detección de movimiento, 
        if detectMove(): 
            #Si detecta movimiento envía el SMS de notificación
            sendSMS()
            #Despues de enviar el SMS se detiene por 10 segundos antes de seguir detectando movimiento
            time.sleep(10)
            #Reinicializa el contador
            counterdetect = 0
        else:
            #Si no detecto movimiento aumenta el contador
            counterdetect += 1
    #Si el modo de DURMIENTE
    else:
        #Empieza a verificar que haya recibido un SMS de notificación para desperar
        if scanSMSs(lastcheck):
            #Si recibe la notificación cambia a modo VIGILANTE e inicializa el contador de detecciones en cero
            modo = 'V'
            counterdetect = 0
    #Si se intento detectar el movimiento mas de 10 veces sin éxito y si el dispositivo NO es el primero de la línea de detección
    if counterdetect > 10 and vecino1:
        #Vuelve a pasar a estado DURMIENTE
        modo = 'D'
    print modo
    #Espera 1 segundo entre operaciones
    time.sleep(1)
print "Program finish"